In [1]:
pip install langchain langchain-openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
import os

# Replace with your actual key or keep as-is if it's already in a variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [10]:
"""Chatbot with OpenAI and LangChain using a custom calculator tool and manual prompt."""

import os
from typing import Dict, List
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent

def simple_calculator(query: str) -> str:
    """
    A simple calculator that evaluates basic math expressions from a string.

    Args:
        query (str): A string representing a math expression (e.g., "2 + 2").

    Returns:
        str: The result of the calculation or an error message.
    """
    try:
        result = eval(query, {"__builtins__": {}})
        return str(result)
    except Exception as err:
        return f"Error in calculation: {err}"


def get_calculator_tool() -> Tool:
    """
    Returns a LangChain Tool instance for the calculator.

    Returns:
        Tool: The calculator tool with metadata and callback function.
    """
    return Tool(
        name="Calculator",
        description="Evaluates basic math expressions (e.g., '3 * (4 + 5)').",
        func=simple_calculator,
    )


def get_llm() -> ChatOpenAI:
    """
    Initializes and returns the OpenAI language model using the environment variable.

    Returns:
        ChatOpenAI: The initialized language model for use in the agent.
    """
    api_key: str | None = os.environ.get("OPENAI_API_KEY")
    if api_key is None:
        raise ValueError("OPENAI_API_KEY environment variable is not set.")
    return ChatOpenAI(model="gpt-4", temperature=0.0)


def get_prompt() -> ChatPromptTemplate:
    """
    Returns a manually constructed ChatPromptTemplate with required variables.

    Returns:
        ChatPromptTemplate: The prompt template including system and user messages.
    """
    return ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that can use tools."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])


def get_agent_executor() -> AgentExecutor:
    """
    Sets up and returns the LangChain AgentExecutor with OpenAI and custom tools.

    Returns:
        AgentExecutor: The configured agent executor.
    """
    llm: ChatOpenAI = get_llm()
    tools: List[Tool] = [
        get_calculator_tool(),
        # get_hotlines_tool()
    ]
    prompt: ChatPromptTemplate = get_prompt()
    agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)


def run_chat_loop(agent_executor: AgentExecutor) -> None:
    """
    Runs a command-line loop for user interaction with the chatbot.

    Args:
        agent_executor (AgentExecutor): The agent executor to process user inputs.
    """
    print("Chatbot is ready. Type 'exit' to quit.")
    while True:
        user_input: str = input("User: ")
        if user_input.lower() == "exit" or user_input.lower() == "quit":
            print("Exiting chatbot.")
            break
        try:
            response: Dict[str, str] = agent_executor.invoke({"input": user_input})
            print("Bot:", response.get("output", "[No output]"))
        except Exception as error:
            print("Error during chat:", error)


In [12]:
executor: AgentExecutor = get_agent_executor()
run_chat_loop(executor)

Chatbot is ready. Type 'exit' to quit.
User: tell me a joke


> Entering new AgentExecutor chain...
Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!

> Finished chain.
Bot: Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!
User: what is the answer of 1-2*19/23


> Entering new AgentExecutor chain...

Invoking: `Calculator` with `1-2*19/23`


-0.6521739130434783The answer to the expression 1-2*19/23 is approximately -0.65.

> Finished chain.
Bot: The answer to the expression 1-2*19/23 is approximately -0.65.
User: exit
Exiting chatbot.
